**Indices**  
j - job  
m - machine 

**Decision Variable**  
$s_{j,m}$ - start time of job j at machine m  
$e_{j,m}$ - end time of job j at machine m  
$d_{j,m}$ - duration of job j at machine m  
$b_{j,m}$ - machine m is selected for job j     
$a_{j,i,m}$ - job i precedes job j at machine m 

**Parameters**  
H - Horizon  
$d_j$ - delivery date of job j  
$q_j$ - order qty of job j  
$o_{j,m}$ - output per hour of job j at machine m  
$set_{i,j}$ - setup time between job i & j   


**Objective** (Not implemented right now)  
$ \sum_{j} d_j - max(e_{j,m})  \qquad \forall m \in mc$    


**Constraints**  
$ \sum_m e_{j,m} < s_{i,m} or s_{j,m} > e_{i,m} \qquad \forall j,i \in jobs$  
$ \sum_j o_{j,m} \times d_{j,m} >= q_j \qquad \forall m \in ma $    
$ \sum_m s_{i,m} >= e_{j,m} + set_{i,j} \qquad if \quad b_{j,m}=1, b_{i,m}=1, a_{j,i,m}=1 $  

In [3]:
!pip install ortools
#!pip install dtale

In [4]:
import pandas as pd
import numpy as np
import collections
from ortools.sat.python import cp_model
# Clone the entire repo.
!git clone -l -s https://github.com/mswastik/optimization.git cloned-repo
%cd cloned-repo

fatal: destination path 'cloned-repo' already exists and is not an empty directory.
/content/cloned-repo


In [5]:
#Importing Data
df=pd.read_excel('orders1.xlsx')
ma=pd.read_excel('item master.xlsx')
bu=pd.read_excel('item master.xlsx',sheet_name='bulk')
pa=pd.read_excel('item master.xlsx',sheet_name='packing')
 
# Formating Data
df = df[df['Line']=='Tmmthpkhti Limi']
df = df[df['Plant']==1024]
del df['Line']
df['due'] = df['Dispatch Date'] - df['SO Date'].min()
df['due'] = df['due'].dt.days
df['sodate'] = df['SO Date'] - df['SO Date'].min()
df['sodate'] = df['sodate'].dt.days
df.reset_index(inplace=True,drop=True)
df['Key']=df['SO'].astype('str')+df['FG Code']
del df['Dispatch Date']
del df['SO Date']
del df['Description']
del df['Customer']
df['Order Qty'] = df[df['Order Qty']!=0]['Order Qty']
df['Order Qty'] = df['Order Qty']*1000
df=df[df['Key']!='17023287gd39000800iX']
 
 
# Creating Parameter Data
jobs =df['Key']
op =[1,2,3,4]  # Ignore as of now
mc = [1,2,3,4,5,6]
#horizon = max(df["due"])+900
due = df.set_index('Key')
dur=df[['Key','FG Code','Order Qty']]
dur=dur.merge(pa[["FG Code","Line","Output"]], on="FG Code")
dur['Output']=dur['Output']*100
dur['duration']=dur['Order Qty']/dur['Output']
dur['duration'] = dur['duration'].astype('float64').round(decimals = 2)
dur.set_index(['Key','Line'],inplace=True)
del dur['FG Code']
dur['Output'].replace(0,1,inplace=True)
df['Order Qty']=df['Order Qty'].fillna(0).astype('int')
dur['duration']=dur['duration'].fillna(0).astype('int')
dur['Output']=dur['Output'].fillna(0).astype('int')
horizon = sum(dur['duration'])
 
#Setup
setup = pd.DataFrame(columns=('Key1','Key2','setup'))
for i in df['Key']:
  for k in df['Key']:
    if df[df['Key']==i]["FG Code"].values[0]==df[df['Key']==k]["FG Code"].values[0]:
      f = {'Key1':[i],"Key2":[k],"setup":[0]}
    else:
      f = {'Key1':[i],"Key2":[k],"setup":[4]}
    f = pd.DataFrame(data=f)
    setup = pd.concat([setup,f])
setup.set_index(['Key1','Key2'],inplace=True)

In [147]:
# Initialise model
model = cp_model.CpModel()

# Creating variables to store data
task_type = collections.namedtuple('task_type', 'start end dur run1')
mtj = collections.defaultdict(list)
all_tasks = {}
 
for j in jobs:
  for m in mc:
      suffix = '_%s_%i' % (j,m)
      start = model.NewIntVar(0, horizon, "start"+suffix)
      end = model.NewIntVar(0, horizon, "end"+suffix)
      run1 = model.NewIntVar(0, dur['duration'].loc[(j,m)].item(), "run"+suffix)
      duration = model.NewIntervalVar(start, run1, end, "duration"+suffix)
      all_tasks[j, m] = task_type(start=start, end=end, dur= duration, run1=run1)
      mtj[m].append(duration)

# Selection variable to store whether machine is selected to process job
b={}
for j in jobs:
  for m in mc:
    b[j,m] = model.NewBoolVar('selection')
    model.Add(all_tasks[j, m].run1>0).OnlyEnforceIf(b[j,m])
    model.Add(all_tasks[j, m].run1==0).OnlyEnforceIf(b[j,m].Not())
    #model.Add(all_tasks[i, m].end==0).OnlyEnforceIf(b[i,m].Not())

# Variable to store sequence of jobs
a={}
for j in jobs:
  for m in mc:
    for i in jobs:
      if j != i:
        a[j,i,m] = model.NewBoolVar('sequence')
        model.Add(all_tasks[i, m].start>all_tasks[j, m].start).OnlyEnforceIf(a[j,i,m])
        model.Add(all_tasks[i, m].start<all_tasks[j, m].start).OnlyEnforceIf(a[j,i,m].Not())

# Constraint: A machine can process only 1 job at a time 
for m in mc:
  model.AddNoOverlap(mtj[m])

# Constraint: Complete production of full order quantity
com = {}
for j in jobs:
  tt= 0
  for m in mc:
    comp = all_tasks[j,m].run1*dur["Output"].loc[(j,m)]
    tt = tt + comp
  com[j] = tt
  model.Add(com[j]*10 >= df[df["Key"]==j]['Order Qty'].values[0])
 
# Constraint of Setup Time     
for j in jobs:
  for m in mc:
    for i in jobs:
      if j != i:
        #model.AddBoolOr([a[j,i,m],a[i,j,m]])
        model.Add(all_tasks[i, m].start >= all_tasks[j, m].end + setup.loc[(j,i)].values[0]).OnlyEnforceIf(a[j,i,m]).OnlyEnforceIf(b[j,m]).OnlyEnforceIf(b[i,m])
        #model.Add(all_tasks[j, m].start >= all_tasks[i, m].end + setup.loc[(j,i)].values[0]).OnlyEnforceIf(a[,i,m].Not()).OnlyEnforceIf(b[j,m]).OnlyEnforceIf(b[i,m])

#kk = model.NewIntVar(-horizon, horizon, 'dummy')
gg={}
#tt={}
for j in jobs:
  tt = due['due'].loc[j]- max([all_tasks[j,m].end for m in mc])
  gg[j] = tt
  #if tt<0:
  #  gg[j] = -tt
  #else:
  #  gg[j] = tt
 
model.Minimize(sum([gg[j] for j in jobs]))
solver = cp_model.CpSolver()
solver.Solve(model)
print(solver.StatusName(),solver.ObjectiveValue())

TypeError: ignored

In [141]:
ff=pd.DataFrame()
for m in mc:
  for j in jobs:
    if solver.Value(all_tasks[j,m].run1)>0:
      kk={'Job':j,'line':m,'start':solver.Value(all_tasks[j,m].start),
          'run':solver.Value(all_tasks[j,m].run1),'end':solver.Value(all_tasks[j,m].end)}
      ff=ff.append(kk,ignore_index=True)
 
#Exploring Result (click on last link to view result)
'''
!pip install dtale
from dtale import show
import dtale.app as dtale_app
dtale_app.USE_COLAB = True
show(ff,ignore_duplicate=True)
'''

'\n!pip install dtale\nfrom dtale import show\nimport dtale.app as dtale_app\ndtale_app.USE_COLAB = True\nshow(ff,ignore_duplicate=True)\n'

In [148]:
tt

SumArray(SumArray(SumArray(SumArray(SumArray(SumArray(ProductCst(run_4501763335gC25020000UC_1(0..2000), 100), 0), ProductCst(run_4501763335gC25020000UC_2(0..2000), 100), 0), ProductCst(run_4501763335gC25020000UC_3(0..2000), 100), 0), ProductCst(run_4501763335gC25020000UC_4(0..2000), 100), 0), ProductCst(run_4501763335gC25020000UC_5(0..2000), 100), 0), ProductCst(run_4501763335gC25020000UC_6(0..66), 3000), 0)

In [142]:
!pip install -U plotly
import plotly.express as px
import datetime
fig=px.timeline(data_frame=ff, x_start=ff['start'].astype('datetime64[h]'),x_end=ff['end'].astype('datetime64[h]'),
                facet_row='line',y='Job',height=1200,width=1200)
fig.update_xaxes(dtick=14400000,tickformat="%d-%H")
#fig.update_yaxes(autorange="reversed")

Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.10.0)


In [143]:
print(model.ModelStats())

Optimization model '':
#Variables: 2040 (17 in objective)
 - 1734 in [0,1]
 - 5 in [0,15]
 - 5 in [0,36]
 - 1 in [0,66]
 - 5 in [0,75]
 - 5 in [0,93]
 - 5 in [0,108]
 - 5 in [0,133]
 - 1 in [0,162]
 - 2 in [0,168]
 - 3 in [0,189]
 - 1 in [0,213]
 - 2 in [0,216]
 - 5 in [0,233]
 - 3 in [0,243]
 - 1 in [0,300]
 - 2 in [0,320]
 - 2 in [0,400]
 - 8 in [0,450]
 - 3 in [0,451]
 - 1 in [0,462]
 - 1 in [0,497]
 - 2 in [0,693]
 - 2 in [0,746]
 - 1 in [0,940]
 - 3 in [0,978]
 - 1 in [0,995]
 - 3 in [0,1054]
 - 6 in [0,1066]
 - 2 in [0,1493]
 - 5 in [0,2000]
 - 3 in [0,2108]
 - 1 in [0,2160]
 - 1 in [0,4500]
 - 1 in [0,5600]
 - 1 in [0,8000]
 - 1 in [0,8640]
 - 1 in [0,14000]
 - 1 in [0,15120]
 - 1 in [0,36000]
 - 204 in [0,1442570]
#kInterval: 102
#kLinear1: 204 (#enforced: 204)
#kLinear2: 4896 (#enforced: 4896)
#kLinearN: 17
#kNoOverlap: 6


In [112]:
import plotly.io as pio
pio.renderers.default = "colab"